In [ ]:
library(SingleCellExperiment); library(DropletUtils); library(zellkonverter); library(celda)

# Load raw and set barcodes
sce_raw_filtered <- DropletUtils::read10xCounts("00_data/raw.h5")

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [ ]:
sce_raw_filtered@rowRanges@elementMetadata

DataFrame with 61313 rows and 3 columns
                          ID                 Symbol            Type
                 <character>            <character>     <character>
1         ENSSSCG00000028996                ALDH1A1 Gene Expression
2         ENSSSCG00000005267                  ANXA1 Gene Expression
3         ENSSSCG00000005268                   RORB Gene Expression
4         ENSSSCG00000005269                  TRPM6 Gene Expression
5         ENSSSCG00000031382                C9orf40 Gene Expression
...                      ...                    ...             ...
61309 AEMK02000695.1:5684-.. AEMK02000695.1:5684-..           Peaks
61310 AEMK02000695.1:9057-.. AEMK02000695.1:9057-..           Peaks
61311 AEMK02000695.1:11026.. AEMK02000695.1:11026..           Peaks
61312 AEMK02000695.1:12452.. AEMK02000695.1:12452..           Peaks
61313 AEMK02000695.1:13456.. AEMK02000695.1:13456..           Peaks

In [ ]:
sce_raw_filtered_filtered <- sce_raw_filtered[sce_raw_filtered@rowRanges@elementMetadata$Type == "Gene Expression"]

In [ ]:
sce_raw_filtered_filtered

class: SingleCellExperiment 
dim: 22040 722431 
metadata(1): Samples
assays(1): counts
rownames(22040): ENSSSCG00000028996 ENSSSCG00000005267 ...
  ENSSSCG00000018094 ENSSSCG00000034846
rowData names(3): ID Symbol Type
colnames: NULL
colData names(2): Sample Barcode
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [9]:
if (is.null(colnames(sce_raw_filtered)) || length(colnames(sce_raw_filtered))==0)
  colnames(sce_raw_filtered) <- as.character(SummarizedExperiment::colData(sce_raw_filtered)[["Barcode"]])

# Foreground barcodes
ad_fg <- zellkonverter::readH5AD("00_data/foreground.h5ad")
norm <- function(x) sub("-1$","", x)
keep_fg <- norm(colnames(sce_raw_filtered)) %in% norm(colnames(ad_fg))
sce_fg <- sce_raw_filtered[, keep_fg, drop=FALSE]

# Read cluster_type produced in Python
lab <- read.csv("51_annot_cells_types.csv", row.names=1, check.names = FALSE)
m <- match(norm(colnames(sce_fg)), norm(rownames(lab)))
sce_fg$cluster_annotation <- factor(lab$cluster_annotation[m])

if (any(is.na(sce_fg$cluster_annotation))) stop("Some barcodes did not match labels. Check CSV/barcodes.")

set.seed(42)
sce_out <- celda::decontX(sce_fg, ,background = sce_raw_filtered, z = "cluster_annotation")

# Save
dir.create("53_results_clusters", showWarnings=FALSE)
HDF5Array::saveHDF5SummarizedExperiment(sce_out, dir="53_results_clusters/decontx_with_clusters_SCEh5", replace=TRUE)
zellkonverter::writeH5AD(sce_out, "53_results_clusters/decontx_with_clusters.h5ad", X_name="decontXcounts")

Wed Sep  3 17:12:02 2025 ..  12315  cells in the background matrix were removed as they were found in  the filtered matrix.

--------------------------------------------------

Starting DecontX

--------------------------------------------------

Wed Sep  3 17:12:03 2025 .. Analyzing all cells

Wed Sep  3 17:12:03 2025 .... Converting to sparse matrix

Wed Sep  3 17:13:18 2025 .... Generating UMAP

Wed Sep  3 17:13:50 2025 .... Estimating contamination

Wed Sep  3 17:13:51 2025 ...... Completed iteration: 10 | converge: 0.02596

Wed Sep  3 17:13:52 2025 ...... Completed iteration: 20 | converge: 0.005476

Wed Sep  3 17:13:52 2025 ...... Completed iteration: 30 | converge: 0.001876

Wed Sep  3 17:13:53 2025 ...... Completed iteration: 38 | converge: 0.0009808

Wed Sep  3 17:13:53 2025 .. Calculating final decontaminated matrix

Wed Sep  3 17:13:54 2025 .. Converting decontaminated matrix to DelayedMatrix

--------------------------------------------------

Completed DecontX. Total time: